# Install and Import Dependencies

In [ ]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np 
import time
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose 

# Pose Detection

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [4]:
def calculate_bar(a,b):
    a = np.array(a) 
    b = np.array(b) 

    bar = (a[1] + b[1])/2

    return bar

In [10]:
cap = cv2.VideoCapture("PU.mp4")
#cap = cv2.VideoCapture(0)

if (cap.isOpened()== False):
    print("Error opening video file")


counter = 0 
hasGoneDown = False
fullyExtended = False
hasStarted = False


## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            wristL = [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
            wristR = [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]

            chin = [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x, landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y]

            shoulderR = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            shoulderL = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

            elbowR = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            elbowL = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate bar
            bar = calculate_bar(wristL, wristR)

            # Calculate angles
            angleL = calculate_angle(shoulderL, elbowL, wristL)
            angleR = calculate_angle(shoulderR, elbowR, wristR)
            
            # Visualize 
            cv2.putText(image, str(bar), 
                           tuple(np.multiply(wristL, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(chin[1]), 
                           tuple(np.multiply(chin, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            cv2.putText(image, str(angleR), 
                           tuple(np.multiply(elbowR, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )

            cv2.putText(image, str(angleL), 
                           tuple(np.multiply(elbowL, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            if not hasStarted:
                cv2.putText(image, 'PRESS S TO START COUNTING', (100,120), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 1, cv2.LINE_AA)
                
            if cv2.waitKey(10) & 0xFF == ord('s'):
                hasStarted = True

            if hasStarted:    
                if chin[1] > bar and not hasGoneDown: 
                    hasGoneDown = True  
                
                if hasGoneDown and angleL > 120 and angleR > 120: 
                    fullyExtended = True  
                
                if chin[1] < bar and hasGoneDown and fullyExtended:  
                    print(f"AngleL: {angleL}, AngleR: {angleR}")
                    counter += 1  
                    print(f"Rep Count: {counter}")
                    
                    hasGoneDown = False  
                    fullyExtended = False 
            
        except:
            pass
        
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        #Draw bar
        # Get image dimensions
        image_height, image_width, _ = image.shape  
        
        # Convert landmarks to pixel coordinates
        wristL = [
            int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x * image_width),
            int(landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y * image_height)
        ]
        
        wristR = [
            int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x * image_width),
            int(landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y * image_height)
        ]
        
        # Convert to tuples
        wristL = tuple(wristL)
        wristR = tuple(wristR)
        
        # Draw the line
        cv2.line(image, wristL, wristR, (255, 0, 230), 9)

        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.putText(image, 'HAS STARTED?', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(hasStarted), 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Pull Up Full ROM', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

AngleL: 41.801160585510246, AngleR: 5.484379437613895
Rep Count: 1
AngleL: 125.28401170757299, AngleR: 121.98966232383
Rep Count: 2
